# Traitement de données textuelles en Python
## Expresssions régulières

In [2]:
import re #On importe la bibliothèqe re, pour regular expressions, qui permet de manipuler des expressions régulières en python

In [ ]:
regex='a' 
s='a'
re.fullmatch(regex,s)#la fonction fullmatch permet de vérifier si une chaîne correspond bien a une expression régulière

In [ ]:
regex='.'
s='B'
re.fullmatch(regex,s)

## Structuration de L'Avare

### Préambule 1
La fonction `finditer` permet de repérer toutes les sous-chaînes correspondant à une expression régulière et de les parcourir:

In [3]:
texte = open('LAvare-re.txt').read()
liste=[]
for personnage in re.finditer("- .*? -", texte):
    liste.append(texte[personnage.start():personnage.end()])
len(liste)

968

### Préambule 2

Les parenthèses dans les expressions régulières définissent des *groupes* auxquels on peut par la suite se référer:

In [ ]:
liste=[]
for personnage in re.finditer("- (.*?) -", texte):
    liste.append(personnage.group(1))
len(liste)

### Préambule 3
Par défaut, en Python, le . correspond à n'importe quel caractère *sauf* le retour à la ligne

In [ ]:
liste=[]
for match in re.finditer(".+", texte):
    liste.append(match)
len(liste)

On peut modifier ce comportement pour travailler sur des expressions régulière qui s'étendent sur plusieurs lignes :

In [ ]:
liste=[]
for match in re.finditer(".+", texte, re.DOTALL):
    liste.append(match)
len(liste)

### Exercice 1
Quelle est l'expression régulière qui permet de décrire un acte ? Quelle est l'expression régulière qui permet de décrire un acte tout en isolant son titre de son texte ?

Vous pouvez tatonner ici : https://regex101.com/r/UZvZCl/3

Ecrire une fonction qui construit un dictionnaire `pièce` qui associe à chaque titre d'acte son texte :

```python
{
    'ACTE PREMIER' : '''Scène première
                        - Valère -
                        Hé quoi ! charmante Élise, vous devenez mélancolique, après les
                        obligeantes assurances que vous avez eu la bonté de me donner de votre
                        foi ? Je vous vois soupire...
                     ''',
    "ACTE SECOND" : """Scène première
                       - Cléante -
                       Ah ! traître que tu es ! où ...
                    """
}
```

In [3]:
pièce={}
for acte in re.finditer('(ACTE .*?)\n(.*?)--ACTE', texte, re.DOTALL):
    titre_acte=acte.group(1)
    texte_acte=acte.group(2)
    pièce[titre_acte]=texte_acte

### Exercice 2
Quelle est l'expression régulière qui permet de décrire une scène ? Quelle est l'expression régulière qui permet de décrire une scène tout en isolant son titre de son texte ?


Ecrire une fonction qui construit un dictionnaire `pièce` qui associe à chaque titre d'acte un dictionnaire qui associe à chaque titre de scène son texte :  

```python
{
    'ACTE PREMIER' : {'Scène première' : """- Valère -
                                            Hé quoi ! charmante Élise, vous devenez mélancolique, après les
                                            obligeantes assurances que vous avez eu la bonté de me donner de votre
                                            foi ? Je vous vois soupire...
                                         """,
                      'Scène II': """- Cléante -
                                     Je suis bien aise de vous trouver seule, ma soeur ; et je brûlais de
                                     vous parler, pour ...
                                     """
                      }
    "ACTE SECOND" : {'Scène première':"""- Cléante -
                                         Ah ! traître que tu es ! où ...
                                      """
                    }
}
```

In [4]:
pièce={}
for acte in re.finditer('(ACTE .*?)\n(.*?)--ACTE', texte, re.DOTALL):
    titre_acte=acte.group(1)
    pièce[titre_acte]={}
    for scène in re.finditer('(Scène .*?)\n(.*?)--Scène', acte.group(2), re.DOTALL):
        titre_scène=scène.group(1)
        texte_scène=scène.group(2)
        pièce[titre_acte][titre_scène]=texte_scène

### Exercice 3
Quelle est l'expression régulière qui permet de décrire une réplique ? Quelle est l'expression régulière qui permet de décrire une scène tout en isolant son personnage de son texte ?

Ecrire une fonction qui construit la structure complète de la pièce :
```python
{
    'ACTE PREMIER' : {'Scène première' : [{'personnage' : 'Valère',
                                           'réplique' : """Hé quoi ! charmante Élise, vous devenez mélancolique, après les
                                                           obligeantes assurances que ...
                                                         """
                                          },
                                          {'personnage' : 'Élise',
                                           'réplique' : """Non, Valère, je ne puis pas me repentir de tout ce que je fais pour
                                                           vous. Je m'y sens entraîner par une trop douce puissance, et je n'ai
                                                           ...
                                                         """
                                          }
                                          ],
                      'Scène II': [{'personnage': 'Cléante',
                                  'réplique': """Je suis bien aise de vous trouver seule, ma soeur ; et je brûlais de
                                                 vous parler, pour m'ouvrir à vous d'un secret.
                                              """
                                  },
                                 {'personnage': 'Élise',
                                  'réplique': "Me voilà prête à vous ouïr, mon frère. Qu'avez-vous à me dire ?"
                                 }
                                 ]
                      }
}
```

In [6]:
pièce={}
for acte in re.finditer('(ACTE .*?)\n(.*?)--ACTE', texte, re.DOTALL):
    titre_acte=acte.group(1)
    pièce[titre_acte]={}
    for scène in re.finditer('(Scène .*?)\n(.*?)--Scène', acte.group(2), re.DOTALL):
        titre_scène=scène.group(1)
        pièce[titre_acte][titre_scène]=[]
        for réplique in re.finditer('- (.*?) -\n(.*?)--REPL', scène.group(2), re.DOTALL):
            pièce[titre_acte][titre_scène].append({'personnage' : réplique.group(1), 'réplique':réplique.group(2)})

In [7]:
pièce['ACTE PREMIER']['Scène IV']

[{'personnage': 'Harpagon',
  'réplique': "Voilà un pendard de valet qui m'incommode fort ; et je ne me plais\npoint à voir ce chien de boiteux-là. Certes, ce n'est pas une petite\npeine que de garder chez soi une grande somme d'argent ; et\nbienheureux qui a tout son fait bien placé, et ne conserve seulement\nque ce qu'il faut pour sa dépense ! On n'est pas peu embarrassé à\ninventer, dans toute une maison, une cache fidèle ; car pour moi, les\ncoffres-forts me sont suspects, et je ne veux jamais m'y fier. Je les\ntiens justement une franche amorce à voleurs, et c'est toujours la\npremière chose que l'on va attaquer.\n"}]

## Analyse des données
Maintenant que l'on a construit notre représention de la pièce, on peut l'exploiter !
### Exercice 4
Constuire la liste des personnages de la pièce
```python
['Valère',
 'Élise',
 'Cléante',
 'Harpagon',
 'La Flèche',
 'Maître Simon',
 'Frosine',
 'Maître Jacques',
 'La Merluche',
 'Brindavoine',
 'Mariane',
 'Le commissaire',
 'Anselme']
```

In [ ]:
personnages=[]
for acte in pièce:
    for scène in pièce[acte]:
        for réplique in pièce[acte][scène]:
            if réplique['personnage'] not in personnages:
                personnages.append(réplique['personnage'])

### Exercice 5
Constuire un dictionnaire qui associe à chaque personnage le nombre de caractères de l'ensemble de ses répliques
```python
{'Anselme': 2740,
 'Brindavoine': 207,
 'Cléante': 17264,
 'Frosine': 12102,
 'Harpagon': 31377,
 'La Flèche': 8494,
 'La Merluche': 289,
 'Le commissaire': 1580,
 'Mariane': 4753,
 'Maître Jacques': 8545,
 'Maître Simon': 971,
 'Valère': 14220,
 'Élise': 5568}
```

In [ ]:
nCar={}
for acte in pièce:
    for scene in pièce[acte]:
        for réplique in pièce[acte][scene]:
            if réplique['personnage'] not in nCar:
                nCar[réplique['personnage']]=0
            nCar[réplique['personnage']]+=len(réplique['réplique'])

### Exercice 6
Constuire un dictionnaire qui associe à chaque personnage une liste de 5 nombres, correspondant au nombre de caractères de leurs répliques, acte par acte
```python
{'Anselme': [0, 0, 0, 0, 2740],
 'Brindavoine': [0, 0, 207, 0, 0],
 'Cléante': [5150, 2592, 3523, 5326, 673],
 'Frosine': [0, 8340, 1520, 2214, 28],
 'Harpagon': [9449, 3796, 6377, 5664, 6091],
 'La Flèche': [1345, 6895, 0, 254, 0],
 'La Merluche': [0, 0, 289, 0, 0],
 'Le commissaire': [0, 0, 0, 0, 1580],
 'Mariane': [0, 0, 2311, 1236, 1206],
 'Maître Jacques': [0, 0, 5032, 1531, 1982],
 'Maître Simon': [0, 971, 0, 0, 0],
 'Valère': [6991, 0, 1951, 0, 5278],
 'Élise': [4384, 0, 98, 331, 755]}
```


In [ ]:
comptesParActe={}
for perso in personnages:
    comptesParActe[perso]=[]
    for acte in pièce:
        comptesParActe[perso].append(0)
        for scène in pièce[acte]:            
            for réplique in pièce[acte][scène]:
                if réplique['personnage']==perso:
                    comptesParActe[perso][-1]+=len(réplique['réplique'])

## Conversion en JSON
JSON est un format de fichier pour enregistré des données structurées très répandu. Nous allons l'utiliser pour conserver notre structuration de la pièce## Bascule dans excel

In [6]:
import json

In [7]:
f=open('LAvare.json', 'w')
f.write(json.dumps(pièce))
f.close()

On peut ensuite recharger les données

In [8]:
f=open('LAvare.json')
pièce = json.loads(f.read())
f.close()

In [9]:
pièce['ACTE PREMIER']['Scène IV']

[{'personnage': 'Harpagon',
  'réplique': "Voilà un pendard de valet qui m'incommode fort ; et je ne me plais\npoint à voir ce chien de boiteux-là. Certes, ce n'est pas une petite\npeine que de garder chez soi une grande somme d'argent ; et\nbienheureux qui a tout son fait bien placé, et ne conserve seulement\nque ce qu'il faut pour sa dépense ! On n'est pas peu embarrassé à\ninventer, dans toute une maison, une cache fidèle ; car pour moi, les\ncoffres-forts me sont suspects, et je ne veux jamais m'y fier. Je les\ntiens justement une franche amorce à voleurs, et c'est toujours la\npremière chose que l'on va attaquer.\n"}]